In [13]:
println(s"Current spark version is ${spark.version}")

Current spark version is 2.4.5


In [14]:
import org.apache.spark.sql.types.{StructType, StructField, IntegerType, LongType, StringType}

val dataSchema = new StructType()
    .add("target", IntegerType)
    .add("id", LongType)
    .add("raw_timestamp", StringType)
    .add("query_status", StringType)
    .add("author", StringType)
    .add("tweet", StringType)

val dataPath= "/home/jovyan/data/training.1600000.processed.noemoticon.csv"

val raw_sentiment = spark.read
    .format("csv")
    .option("header",false)
    .schema(dataSchema)
    .load(dataPath)
    .selectExpr("(case when target=4 then 1 else 0 end) as label","tweet")

raw_sentiment.groupBy($"label").count.show

+-----+------+
|label| count|
+-----+------+
|    1|800000|
|    0|800000|
+-----+------+



dataSchema = StructType(StructField(target,IntegerType,true), StructField(id,LongType,true), StructField(raw_timestamp,StringType,true), StructField(query_status,StringType,true), StructField(author,StringType,true), StructField(tweet,StringType,true))
dataPath = /home/jovyan/data/training.1600000.processed.noemoticon.csv
raw_sentiment = [label: int, tweet: string]


[label: int, tweet: string]

In [15]:
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.RandomForestClassifier
import org.apache.spark.ml.feature.{HashingTF, Tokenizer}
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.sql.Row

val tokenizer = new Tokenizer()
    .setInputCol("tweet")
    .setOutputCol("words")

val hashingTF = new HashingTF()
    .setNumFeatures(1000)
    .setInputCol(tokenizer.getOutputCol)
    .setOutputCol("features")

val lr = new RandomForestClassifier()
    .setNumTrees(25)

val pipeline = new Pipeline()
  .setStages(Array(tokenizer, hashingTF, lr))


tokenizer = tok_fdd4d8a8a26f
hashingTF = hashingTF_313d23af2f6d
lr = rfc_b5e3021a3722
pipeline = pipeline_49f80b3c778d


pipeline_49f80b3c778d

In [16]:
val model = pipeline.fit(raw_sentiment)

model = pipeline_49f80b3c778d


pipeline_49f80b3c778d

In [17]:
model.write.overwrite().save("/home/jovyan/models/spark-ml-model")

In [18]:
val sameModel = PipelineModel.load("/home/jovyan/models/spark-ml-model")

sameModel = pipeline_49f80b3c778d


pipeline_49f80b3c778d

In [19]:
val predictionsDF = sameModel.transform(raw_sentiment)

predictionsDF.show()

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|label|               tweet|               words|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|    0|@switchfoot http:...|[@switchfoot, htt...|(1000,[7,14,21,54...|[11.3547360319872...|[0.45418944127949...|       1.0|
|    0|is upset that he ...|[is, upset, that,...|(1000,[170,193,22...|[12.8595012501999...|[0.51438005000799...|       0.0|
|    0|@Kenichan I dived...|[@kenichan, i, di...|(1000,[10,36,77,1...|[14.2995424574909...|[0.57198169829963...|       0.0|
|    0|my whole body fee...|[my, whole, body,...|(1000,[82,191,296...|[12.9355856535216...|[0.51742342614086...|       0.0|
|    0|@nationwideclass ...|[@nationwideclass...|(1000,[18,96,130,...|[13.2838856656410...|[0.53135542662564...|       0.0|
|    0|@

predictionsDF = [label: int, tweet: string ... 5 more fields]


[label: int, tweet: string ... 5 more fields]

In [20]:
// predictionsDF.schema
// predictionsDF.describe()
// predictionsDF.stat
predictionsDF.printSchema()

root
 |-- label: integer (nullable = false)
 |-- tweet: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [21]:
import org.apache.spark.sql.functions._

val getProbability = udf((prediction: org.apache.spark.ml.linalg.Vector) => prediction(1))

getProbability = UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))


UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))

In [22]:
predictionsDF.select(getProbability($"probability").alias("clean_probability")).show

+-------------------+
|  clean_probability|
+-------------------+
| 0.5458105587205082|
| 0.4856199499920029|
|  0.428018301700363|
|0.48257657385913383|
|0.46864457337435944|
|  0.501700875084444|
| 0.5165007188948568|
| 0.5547691722494736|
| 0.4827118918311364|
| 0.5189556745544138|
| 0.5189556745544138|
|0.46984949128820264|
|0.45791389822658635|
| 0.5271187032221742|
| 0.4155807683677547|
|0.48338547537275417|
| 0.4897148243594257|
| 0.5121074137672206|
| 0.5728339900273237|
| 0.5721788830172626|
+-------------------+
only showing top 20 rows

